## Task 3: Building Summary Tables
It is useful to have summary files that allow you to quickly answer questions such as the following:
•	How have our sales-by-day changed over the last few months?
•	What is our most popular item in each department?
•	Which owners spend the most per month in each department?

The classic way to structure data to answer these questions is in a relational database. In this task, you will build the summary text files that hold this data and populate a relational database with the data.

Deliverable: The Python code that creates the summary tables. The Python code that builds the database. The actual database file (.db).

### Connect to GBQ

In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
import os

path_to_files = 'C:\\Users\\meand\\Documents\\Grad School\\Fall2020\\Applied Data Analytics\\Wedge\\ada-wedge\\bigwedge_clean\\'

clean_wedge = os.listdir("bigwedge_clean")

#specific to my computer & project
service_path = "C:\\Users\meand\\Documents\\Grad School\Fall2020\\Applied Data Analytics\\Wedge\\ada-wedge\\"
service_file = 'Wedge Project-ed75ab1189b3.json' # this is your authentication information  
gbq_proj_id = 'wedge-project-290522'  # change this to your project_id
gbq_dataset_id = 'big_wedge_1' # and change this to your data set ID

private_key =service_path + service_file

# pass in our credentials so that Python has permission to access our project
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

# establish our connection
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [2]:
#Create a database file

import sqlite3
db = sqlite3.connect("WedgeTask3.db")
cur = db.cursor()

### Query 1:

Sales by date by hour: 
--By calendar date (YYYY-MM-DD) and hour of the day, determine the total spend in the store, the number of transactions, and a count of the number of items .

In [6]:
query_1 = (
    """Select (EXTRACT(date FROM datetime)) AS Date,
    (EXTRACT(hour FROM datetime)) AS Hour,
    SUM(total) AS Sales,
    Count(DISTINCT(Date(datetime) || register_no || emp_no || trans_no)) AS Transactions,
    SUM(CASE WHEN(trans_status = 'V' OR trans_status = 'R') THEN -1 ELSE 1 END) as Items
    FROM `wedge-project-290522.big_wedge_1.transArchive*`
    Where card_no != 3
    AND department != 0 
    AND department != 15
    AND trans_status != 'M'
    AND trans_status != 'C'
    AND trans_status != 'J'
    AND (trans_status = ''
    OR trans_status = ' '
    or trans_status = 'V'
    or trans_status = 'R')
    GROUP BY Date, Hour
    ORDER BY Date, Hour"""
)
    
    
#execute the query
results_1 = client.query(query_1,location = "US",)

In [7]:
#write to a txt file for the first query

with open ('Sales_date_hour.txt', 'w') as outfile:
    for line in results_1:
        row = [str(item) for item in line]
        outfile.write(",".join(row) + "\n")

In [8]:
# input the results of the first query into the WedgeTask3 database as the table Sales_by_date_by_hour
input_file1 = "Sales_date_hour.txt"

db = sqlite3.connect("WedgeTask3.db") # connect to the new database
cur = db.cursor()

cur.execute('''DROP TABLE IF EXISTS Sales_by_Date_by_Hour''') # remove the table if it already exists

# create the table in the connected database and set up the schema
cur.execute('''CREATE TABLE Sales_by_Date_by_Hour (
    Date TIMESTAMP,
    Hour TIMESTAMP,
    Sales REAL,
    Transactions INTEGER,
    Items TEXT)''')

# input the Sales by Date by Hour text data into the established table in the database
with open(input_file1, 'r', encoding = 'Latin-1') as infile:
    for idx,line in enumerate(infile.readlines()):
        line = line.strip().split(',')
        cur.execute('''
        INSERT INTO Sales_by_Date_by_Hour (Date, Hour, Sales, Transactions, Items)
        VALUES (?,?,?,?,?)''', line) #make sure that the ? match up with the column labels for value insertion
db.commit()

### Query 2: 

Sales by owner by year by month: A file that has the following columns: card_no, year, month, sales, transactions, and items.

In [12]:
query2 = (
    """SELECT card_no As Owner,
    (EXTRACT(year FROM datetime)) AS Year,
    (EXTRACT(month FROM datetime)) AS Month,
    SUM(total) AS Sales,
    COUNT(DISTINCT(Date(datetime) || register_no || emp_no || trans_no)) AS Transactions,
    SUM(CASE WHEN(trans_status = 'V' OR trans_status = 'R') THEN -1 ELSE 1 END) as Items
    FROM `wedge-project-290522.big_wedge_1.transArchive*`
    WHERE card_no != 3
    AND department != 0
    AND department != 15
    AND trans_status != 'M'
    AND trans_status != 'C'
    AND trans_status != 'J'
    AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status =  'V'
    OR trans_status = 'R')
    GROUP BY Owner, Year, Month
    ORDER BY Owner, Year, Month DESC"""
)

# execute the query
results2 = client.query(query2, location="US",)

In [13]:
#Create output txt file for the second query

with open('Sales_by_Owner_Date.txt',"w") as outfile:
    
    for row in results2: 
        outfile.write(",".join([str(item) for item in row])+"\n")

In [36]:
# input the results of the second query into the WedgeTask3 database as the table Sales_by_Owner_Date
input_file2 = "Sales_by_Owner_Date.txt"

db = sqlite3.connect("WedgeTask3.db") # connect to the WedgeTask 3 database
cur = db.cursor()

cur.execute('''DROP TABLE IF EXISTS Sales_by_Owner_Date''') # remove the table if it already exists

# create the table in the connected database and set up the schema
cur.execute('''CREATE TABLE Sales_by_Owner_Date (
    Owner INTEGER,
    Year TIMESTAMP,
    Month TIMESTAMP,
    Sales REAL,
    Transactions INTEGER,
    Items TEXT)''')

# input the Sales by Owner by Date text data into the established table in the database
with open(input_file2, 'r', encoding = 'Latin-1') as infile:
    for idx,line in enumerate(infile.readlines()):
        line = line.strip().split(',')
        cur.execute('''
        INSERT INTO Sales_by_Owner_Date (Owner, Year, Month, Sales, Transactions, Items)
        VALUES (?,?,?,?,?,?)''', line)
db.commit()

### Query 3:

Sales by product description by year by month: A file that has the following columns: upc (universal product code), description, department number, department name, year, month, sales, transactions, and items.

In [32]:
query3 = (
    """SELECT Upc AS UPC,
    description AS Product,
    a.department AS Department,
    b.dept_name AS Dept_Name,
    (EXTRACT(year FROM datetime)) AS Year,
    (EXTRACT(month FROM datetime)) AS Month,
    SUM(total) AS Sales,
    COUNT(DISTINCT(Date(datetime) || register_no || emp_no || trans_no)) AS Transactions,
    SUM(CASE WHEN(trans_status = 'V' OR trans_status = 'R') THEN -1 ELSE 1 END) as Items
    FROM `big_wedge_1.transArchive*` a
    LEFT OUTER JOIN `big_wedge_1.department` b
    ON a.department = b.department
    WHERE card_no != 3
    AND a.department != 0
    AND a.department != 15
    AND trans_status != 'M'
    AND trans_status != 'C'
    AND trans_status != 'J'
    AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status =  'V'
    OR trans_status = 'R')
    GROUP BY UPC, Product, Department, Dept_Name, Year, Month
    ORDER BY UPC, Product, Department, Dept_Name, Year, Month DESC"""
)

# Execute the query
results3 = client.query(query3,location="US",)

In [33]:
# output the thrid query results as a text file
with open('Sales_by_Product_by_Date.txt', 'w') as outfile :
    for row in results3: 
        outfile.write(",".join([str(item) for item in row])+"\n")

In [34]:
# input the results of the first query into the WedgeTask3 database as the table Sales_by_Product_by_Date
input_file3 = "Sales_by_Product_by_Date.txt"

db = sqlite3.connect("WedgeTask3.db")# connect to the WedgeTask 3 database
cur = db.cursor()

cur.execute('''DROP TABLE IF EXISTS Sales_by_Product_by_Date''') # remove the table if it already exists

# create the table in the connected database and set up the schema
cur.execute('''CREATE TABLE Sales_by_Product_by_Date (
    UPC STRING,
    Description STRING,
    Department FLOAT,
    Dept_Name STRING,
    Year TIMESTAMP,
    Month TIMESTAMP,
    Sales REAL,
    Transactions INTEGER,
    Items TEXT)''')


In [35]:
# input the Sales by Product by Date text data into the established table in the database
with open(input_file3, 'r', encoding = 'Latin-1') as infile:
    for idx,line in enumerate(infile.readlines()):
        line = line.strip().split(',')
        cur.execute('''
        INSERT INTO Sales_by_Product_by_Date (UPC, Description, Department, Dept_Name, Year, Month, Sales, Transactions, Items)
        VALUES (?,?,?,?,?,?,?,?,?)''', line)
db.commit()